#Importing Colab and necessary packages

In [2]:
from google.colab import drive
drive.mount("/content/drive")

Mounted at /content/drive


In [3]:
!pip install torchtext -qq
!pip install transformers evaluate wandb datasets accelerate -U -qq
!pip install emoji cleantext -qq

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 8.8/8.8 MB 34.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 84.1/84.1 kB 12.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.2/2.2 MB 70.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 510.5/510.5 kB 3.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 297.4/297.4 kB 38.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 16.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 27.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 19.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.3/207.3 kB 28.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 267.1/267.1 kB 31.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.7/62.7 kB 9.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 433.8/433.8 kB 9.0 MB/s eta 0:00:00


In [4]:
import random
import numpy as np
import joblib
from collections import Counter
from pathlib import Path
import sys
import pandas as pd
from sklearn.model_selection import train_test_split
from datasets import Dataset
import base64
from transformers.modeling_outputs import SequenceClassifierOutput
from transformers import PreTrainedModel, PretrainedConfig
from transformers import TrainingArguments, Trainer
from functools import partial
import wandb
import matplotlib.pyplot as plt
from datasets import load_dataset, DatasetDict
from transformers import TrainingArguments, Trainer
from transformers import AutoTokenizer
from transformers import AutoModelForSequenceClassification
from transformers import AutoConfig
from transformers import pipeline
import re
from cleantext import clean
import emoji
from datasets import load_dataset

##Path and defining folders

In [ ]:
#google/flan-t5-base

In [8]:
basepath = ("/content/drive/MyDrive/Applied_NLP/")
sys.path.append("/content/drive/MyDrive/Applied_NLP/Custom_Files/")
sys.path

['/content',
 '/env/python',
 '/usr/lib/python310.zip',
 '/usr/lib/python3.10',
 '/usr/lib/python3.10/lib-dynload',
 '',
 '/usr/local/lib/python3.10/dist-packages',
 '/usr/lib/python3/dist-packages',
 '/usr/local/lib/python3.10/dist-packages/IPython/extensions',
 '/root/.ipython',
 '/tmp/tmp74ol5nrv',
 '/content/drive/MyDrive/Applied_NLP/Custom_Files/']

In [9]:
basefolder = Path(basepath)
datafolder = basefolder/"Class_7/Home_Work_5/Data/"
modelfolder = basefolder/"Class_8/Home_Work_6/Model_Checkpoint/glan-without-preprocessing"
customfolder = basefolder/"Custom_Files/"

In [ ]:
from CustomPreprocessorSpacy import SpacyPreprocessor

In [ ]:
datafolder

PosixPath('/content/drive/MyDrive/Applied_NLP/Class_7/Home_Work_5/Data')

#Importing Data

In [19]:
df_train = pd.read_csv(datafolder/"emoticons_train.csv")

In [20]:
df_test = pd.read_csv(datafolder/"emoticons_test.csv")

##Labels in the data

In [21]:
labels = [label for label in df_train.columns if label not in ["Tweet","ID"]]
id2labels = {idx:label for idx, label in enumerate(labels)}
labels2id = {v:k for k,v in id2labels.items()}
labels2id

{'anger': 0,
 'anticipation': 1,
 'disgust': 2,
 'fear': 3,
 'joy': 4,
 'love': 5,
 'optimism': 6,
 'pessimism': 7,
 'sadness': 8,
 'surprise': 9,
 'trust': 10}

#Dropping,Preprocessing and Splitting

In [22]:
df_train.drop("ID",axis=1,inplace=True)
df_test.drop("ID", axis=1, inplace=True)

In [ ]:
#df_train["Tweet"] = [emoji.replace_emoji(text, replace='') for text in df_train["Tweet"]]
#df_train["Tweet"] = [re.sub(r'@\w+|#\w+', '', text)for text in df_train["Tweet"]]
#df_test["Tweet"] = [emoji.replace_emoji(text, replace='') for text in df_test["Tweet"]]
#df_test["Tweet"] = [re.sub(r'@\w+|#\w+', '', text)for text in df_test["Tweet"]]

In [ ]:
df_train

,Tweet,anger,anticipation,disgust,fear,joy,love,optimism,pessimism,sadness,surprise,trust
0,“Worry is a down payment on a problem you may ...,0,1,0,0,0,0,1,0,0,0,1
1,Whatever you decide to do make sure it makes y...,0,0,0,0,1,1,1,0,0,0,0
2,@Max_Kellerman it also helps that the majorit...,1,0,1,0,1,0,1,0,0,0,0
3,Accept the challenges so that you can literall...,0,0,0,0,1,0,1,0,0,0,0
4,My roommate: it's okay that we can't spell bec...,1,0,1,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...
7719,@BadHombreNPS @SecretaryPerry If this didn't m...,1,0,1,0,0,0,0,0,0,0,0
7720,Excited to watch #stateoforigin tonight! Come ...,0,0,0,0,1,0,1,0,0,0,0
7721,"Blah blah blah Kyrie, IT, etc. @CJC9BOSS leavi...",1,0,1,0,0,0,0,0,1,0,0
7722,#ThingsIveLearned The wise #shepherd never tru...,0,0,0,0,0,0,0,0,0,0,0


##Spacy Preprocessing

In [ ]:
sp = SpacyPreprocessor(model='en_core_web_sm', batch_size=64, lemmatize=True, lower=True,
                                    remove_stop=True, remove_punct=True, remove_email=True,
                                    remove_url=True, remove_num=False, stemming=False,
                                    add_user_mention_prefix=False, remove_hashtag_prefix=True, basic_clean_only=False)

In [23]:
X_train_total = df_train["Tweet"].tolist()

In [24]:
X_test = df_test["Tweet"].tolist()

In [25]:
X_train_total

["“Worry is a down payment on a problem you may never have'. \xa0Joyce Meyer.  #motivation #leadership #worry",
 'Whatever you decide to do make sure it makes you #happy.',
 "@Max_Kellerman  it also helps that the majority of NFL coaching is inept. Some of Bill O'Brien's play calling was wow, ! #GOPATS",
 "Accept the challenges so that you can literally even feel the exhilaration of victory.' -- George S. Patton 🐶",
 "My roommate: it's okay that we can't spell because we have autocorrect. #terrible #firstworldprobs",
 "No but that's so cute. Atsu was probably shy about photos before but cherry helped her out uwu",
 'Do you think humans have the sense for recognizing impending doom? ',
 "Rooneys fucking untouchable isn't he? Been fucking dreadful again, depay has looked decent(ish)tonight",
 "it's pretty depressing when u hit pan on ur favourite highlighter",
 '@BossUpJaee but your pussy was weak from what I heard so stfu up to me bitch . You got to threaten him that your pregnant .',
 

In [26]:
y_test = df_train

In [27]:
y_train_total = df_train.drop("Tweet",axis=1).to_numpy().astype(float)
y_test = df_test.drop("Tweet",axis=1).to_numpy()

In [28]:
y_train_total

array([[0., 1., 0., ..., 0., 0., 1.],
       [0., 0., 0., ..., 0., 0., 0.],
       [1., 0., 1., ..., 0., 0., 0.],
       ...,
       [1., 0., 1., ..., 1., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.]])

In [29]:
y_test = np.where(y_test == "NONE",0,y_test).astype(float)

##Train-Valid Split

In [30]:
X_train,X_valid,y_train,y_valid = train_test_split(X_train_total,y_train_total,test_size=0.3, random_state=42, shuffle=True)

##Spacy preprocessor

In [ ]:
#X_train_preprocessed = sp.fit_transform(X_train)
#X_valid_preprocessed = sp.transform(X_valid)
#X_test_preprocessed = sp.transform(X_test)

##Data Dictionaries

In [31]:
trainset = Dataset.from_dict({
    'texts': X_train,
    'labels': y_train
})

validset = Dataset.from_dict({
    'texts': X_valid,
    'labels': y_valid
})

testset = Dataset.from_dict({
    'texts': X_test,
    'labels': y_test
})

In [32]:
trainset[0:5]

{'texts': ['@KMunie7 @KaranEsch Helluva lot more animated than they were for the actual game &gt;.&lt;',
  "Despite my sharing O'Neill's article, what saddens me about the Gazza case is there seems to be more fury about the sentence than the remark",
  '@mtailor “20% Better Than a Tailor”? Least a tailor would know #navy vs. #blue. $500 and 4 weeks for an #innacurate suit? ',
  'How on earth can the projection of all that is good and happy and true in my soul be a poisonous fucking snake? #patronus #adder #offended',
  'Patti seems so sad. She stamped and ran behind the sofa. We will have to give her plenty of love and affection...more than usual. '],
 'labels': [[0.0, 1.0, 0.0, 0.0, 1.0, 1.0, 1.0, 0.0, 0.0, 0.0, 0.0],
  [1.0, 0.0, 1.0, 1.0, 0.0, 0.0, 0.0, 0.0, 1.0, 0.0, 0.0],
  [1.0, 0.0, 1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 1.0, 0.0, 0.0],
  [1.0, 0.0, 1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0],
  [0.0, 0.0, 0.0, 0.0, 1.0, 1.0, 1.0, 0.0, 1.0, 0.0, 0.0]]}

In [33]:
testset[0:5]

{'texts': ['@Adnan__786__ @AsYouNotWish Dont worry Indian army is on its ways to dispatch all Terrorists to Hell',
  'Academy of Sciences, eschews the normally sober tone of scientific papers and calls the massive loss of wildlife a “biological annihilation',
  'I blew that opportunity -__- #mad',
  'This time in 2 weeks I will be 30... 😥',
  '#Deppression is real. Partners w/ #depressed people truly dont understand the depth in which they affect us. Add in #anxiety &amp;makes it worse'],
 'labels': [[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0],
  [0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0],
  [0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0],
  [0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0],
  [0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0]]}

In [34]:
trainset.features

{'texts': Value(dtype='string', id=None),
 'labels': Sequence(feature=Value(dtype='float64', id=None), length=-1, id=None)}

#Distil-bert as Checkpoint

In [13]:
checkpoint_google_flan = "google/flan-t5-base"
tokenizer_google_flan = AutoTokenizer.from_pretrained(checkpoint_google_flan)

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:88: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/2.54k [00:00<?, ?B/s]

spiece.model:   0%|          | 0.00/792k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/2.42M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/2.20k [00:00<?, ?B/s]

##Tokenize for distil-bert



In [14]:
def tokenize_fn_googleflan(batch):
  return tokenizer_google_flan(text = batch["texts"], truncation=True, padding="max_length",return_tensors = "pt", max_length = 128)

In [35]:
tokenized_dataset_train_gflan = trainset.map(tokenize_fn_googleflan,batched=True)
tokenized_dataset_valid_gflan = validset.map(tokenize_fn_googleflan,batched=True)
tokenized_dataset_test_gflan = testset.map(tokenize_fn_googleflan,batched=True)

Map:   0%|          | 0/5406 [00:00<?, ? examples/s]

Map:   0%|          | 0/2318 [00:00<?, ? examples/s]

Map:   0%|          | 0/3259 [00:00<?, ? examples/s]

In [36]:
tokenized_dataset_train_gflan

Dataset({
    features: ['texts', 'labels', 'input_ids', 'attention_mask'],
    num_rows: 5406
})

In [37]:
tokenized_dataset_train_gflan = tokenized_dataset_train_gflan.remove_columns(["texts"])
tokenized_dataset_valid_gflan = tokenized_dataset_valid_gflan.remove_columns(["texts"])
tokenized_dataset_test_gflan = tokenized_dataset_test_gflan.remove_columns(["texts"])

In [38]:
tokenized_dataset_train_gflan

Dataset({
    features: ['labels', 'input_ids', 'attention_mask'],
    num_rows: 5406
})

In [39]:
tokenized_dataset_train_gflan.set_format(type='torch')
tokenized_dataset_valid_gflan.set_format(type='torch')
tokenized_dataset_test_gflan.set_format(type='torch')

In [40]:
tokenized_dataset_train_gflan

Dataset({
    features: ['labels', 'input_ids', 'attention_mask'],
    num_rows: 5406
})

In [41]:
tokenized_dataset_test_gflan

Dataset({
    features: ['labels', 'input_ids', 'attention_mask'],
    num_rows: 3259
})

##Model for distil-bert

In [ ]:
model_gflan = AutoModelForSequenceClassification.from_pretrained(checkpoint_google_flan,num_labels = 11,problem_type="multi_label_classification",id2label=id2labels,label2id=labels2id)

config.json:   0%|          | 0.00/1.40k [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/990M [00:00<?, ?B/s]

Some weights of T5ForSequenceClassification were not initialized from the model checkpoint at google/flan-t5-base and are newly initialized: ['classification_head.dense.bias', 'classification_head.dense.weight', 'classification_head.out_proj.bias', 'classification_head.out_proj.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [5]:
import evaluate
from transformers import EvalPrediction
import torch
from sklearn.metrics import f1_score, roc_auc_score, accuracy_score

##Metrics labelling functions

In [ ]:
def multi_label_metrics(predictions, labels, threshold=0.5):
    sigmoid = torch.nn.Sigmoid()
    probs = sigmoid(torch.Tensor(predictions))
    y_pred = np.zeros(probs.shape)
    y_pred[np.where(probs >= threshold)] = 1
    y_true = labels
    f1_micro_average = f1_score(y_true=y_true, y_pred=y_pred, average='micro')
    accuracy = accuracy_score(y_true, y_pred)
    metrics = {'f1': f1_micro_average,'Accuracy': accuracy}
    return metrics

def compute_metrics_1(p: EvalPrediction):
    preds = p.predictions[0] if isinstance(p.predictions,tuple) else p.predictions
    result = multi_label_metrics(predictions=preds,labels=p.label_ids)
    return result

##Training Args

In [ ]:
training_args = TrainingArguments(
    num_train_epochs=5,
    per_device_train_batch_size=8,
    per_device_eval_batch_size=8,
    weight_decay=0.01,
    learning_rate=0.00002,
    optim='adamw_torch',
    remove_unused_columns=False,
    output_dir=str(modelfolder),
    evaluation_strategy='steps',
    eval_steps=100,
    save_strategy="steps",
    save_steps=100,
    load_best_model_at_end=True,
    save_total_limit=2,
    metric_for_best_model="f1",
    greater_is_better=True,
    logging_strategy='steps',
    logging_steps=10,
    report_to='wandb',
    run_name='emoticon-gflan-without-preprocessing',
)

##G-FLAN Trainer

In [ ]:
trainer_gflan = Trainer(
    model=model_gflan,
    args=training_args,
    train_dataset = tokenized_dataset_train_gflan,
    eval_dataset = tokenized_dataset_valid_gflan,
    compute_metrics=compute_metrics_1,
)


/usr/local/lib/python3.10/dist-packages/accelerate/accelerator.py:436: FutureWarning: Passing the following arguments to `Accelerator` is deprecated and will be removed in version 1.0 of Accelerate: dict_keys(['dispatch_batches', 'split_batches', 'even_batches', 'use_seedable_sampler']). Please pass an `accelerate.DataLoaderConfiguration` instead: 
dataloader_config = DataLoaderConfiguration(dispatch_batches=None, split_batches=False, even_batches=True, use_seedable_sampler=True)
  warnings.warn(


In [ ]:
!wandb login 5b0f2851a77f589c556f16598a0b181257a541e9

wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc


In [ ]:
%env WANDB_PROJECT = nlp_course_spring_2024-emtoicons-analysis-hf-trainer

env: WANDB_PROJECT=nlp_course_spring_2024-emtoicons-analysis-hf-trainer


##Training

In [ ]:
trainer_gflan.train()

wandb: Currently logged in as: abhishekmoguluri (abhishek-moguluri). Use `wandb login --relogin` to force relogin


Step,Training Loss,Validation Loss,F1,Accuracy
100,0.471800,0.473395,0.013391,0.023296
200,0.461600,0.467663,0.000000,0.023727
300,0.445600,0.452767,0.100000,0.030630
400,0.432300,0.411280,0.435764,0.100518
500,0.366600,0.383619,0.571056,0.160483
600,0.409500,0.372522,0.591343,0.164797
700,0.387600,0.360047,0.600617,0.189819
800,0.379700,0.360145,0.603815,0.191113
900,0.362000,0.353844,0.601648,0.194564
1000,0.338700,0.351292,0.615620,0.200173


There were missing keys in the checkpoint model loaded: ['transformer.encoder.embed_tokens.weight', 'transformer.decoder.embed_tokens.weight'].


TrainOutput(global_step=3380, training_loss=0.35104215667092586, metrics={'train_runtime': 1458.9057, 'train_samples_per_second': 18.528, 'train_steps_per_second': 2.317, 'total_flos': 4127468669130240.0, 'train_loss': 0.35104215667092586, 'epoch': 5.0})

In [ ]:
best_model_gflan_checkpoint_step = trainer_gflan.state.best_model_checkpoint.split('-')[-1]
print(f"The best model was saved at step {best_model_gflan_checkpoint_step}.")

The best model was saved at step 3000.


In [ ]:
wandb.finish()

eval/Accuracy,▁▁▁▃▅▆▆▆▇▇▇▇▇▇▇▇▇██▇█████████████
eval/f1,▁▁▂▆▇▇▇▇▇▇███████████████████████
eval/loss,██▇▅▄▃▃▃▂▂▂▂▂▂▂▂▂▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁
eval/runtime,█▂▁▁▃▂▃▃▃▃▂▁▁▁▁▂▃▂▂▁▁▁▂▂▂▁▂▁▂▂▂▄▃
eval/samples_per_second,▁▇██▆▇▆▆▆▆▇███▇▇▆▇▇███▇▇▇█▇█▇▇▇▅▆
eval/steps_per_second,▁▇██▆▇▆▆▆▆▇███▇▇▆▇▇███▇▇▇█▇█▇▇▇▅▆
train/epoch,▁▁▁▂▂▂▂▂▂▃▃▃▃▃▄▄▄▄▄▄▅▅▅▅▅▅▆▆▆▆▆▇▇▇▇▇▇███
train/global_step,▁▁▁▂▂▂▂▂▂▃▃▃▃▃▄▄▄▄▄▄▅▅▅▅▅▅▆▆▆▆▆▇▇▇▇▇▇███
train/grad_norm,▁▁▂▁▂▂▃▁▂▅▂▂▂▂▂▂▂▃▂▂▂▂▃▂▂▁▂▂▅▁▂▃█▂▃▃▂▂▂▃
train/learning_rate,███▇▇▇▇▇▇▆▆▆▆▆▆▅▅▅▅▅▄▄▄▄▄▄▃▃▃▃▃▃▂▂▂▂▂▁▁▁
train/loss,█▅▅▅▅▄▄▄▄▃▃▂▃▂▂▂▂▃▃▂▂▂▃▃▂▂▂▃▂▁▂▁▂▂▂▂▁▂▂▂


##Save the model

In [18]:
device = "cpu"

In [11]:
#model_gflan = str(modelfolder/f'checkpoint-{best_model_gflan_checkpoint_step}')
model_gflan = str(modelfolder/f'checkpoint-3000')
model_gflan

'/content/drive/MyDrive/Applied_NLP/Class_8/Home_Work_6/Model_Checkpoint/glan-without-preprocessing/checkpoint-3000'

In [12]:
model_gflan = AutoModelForSequenceClassification.from_pretrained(model_gflan)

In [42]:
model_gflan(input_ids=tokenized_dataset_train_gflan['input_ids'][:2].to(device), labels=tokenized_dataset_train_gflan['labels'][:2].to(device))

Seq2SeqSequenceClassifierOutput(loss=tensor(0.3720, grad_fn=<BinaryCrossEntropyWithLogitsBackward0>), logits=tensor([[-3.6599, -0.8917, -3.4487, -3.1517,  2.5832, -1.8799, -0.3087, -4.4910,
         -3.7443, -2.5507, -2.8863],
        [ 1.8267, -3.1414,  1.6853, -2.1518, -3.5536, -5.2189, -3.7556, -2.3501,
         -0.4124, -3.4093, -4.7362]], grad_fn=<AddmmBackward0>), past_key_values=None, decoder_hidden_states=None, decoder_attentions=None, cross_attentions=None, encoder_last_hidden_state=tensor([[[-0.1171,  0.1274, -0.1257,  ...,  0.0253,  0.1286, -0.0694],
         [-0.1049,  0.1244, -0.2414,  ...,  0.0590,  0.0502,  0.0181],
         [-0.0927,  0.0283, -0.1572,  ...,  0.0978,  0.0600, -0.2120],
         ...,
         [-0.0491, -0.0157, -0.1286,  ...,  0.0608,  0.0016,  0.1448],
         [-0.0143, -0.0475, -0.1886,  ...,  0.1212, -0.0516,  0.1491],
         [-0.0018, -0.0487, -0.1517,  ...,  0.1154, -0.0279,  0.1652]],

        [[-0.2402,  0.2407, -0.0650,  ...,  0.1420,  0.0640, 

##Validation set

In [ ]:
training_args_gflan_test = TrainingArguments(
    output_dir=str(modelfolder),
    per_device_eval_batch_size=8,
    do_train=False,
    do_eval=True,
    report_to='wandb',
    run_name='emoticon_hf_trainer',
)

In [ ]:
trainer_gflan_test = Trainer(
    model=model_gflan,
    args=training_args_gflan_test,
    eval_dataset=tokenized_dataset_test_gflan,
    compute_metrics=compute_metrics_1,)

/usr/local/lib/python3.10/dist-packages/accelerate/accelerator.py:436: FutureWarning: Passing the following arguments to `Accelerator` is deprecated and will be removed in version 1.0 of Accelerate: dict_keys(['dispatch_batches', 'split_batches', 'even_batches', 'use_seedable_sampler']). Please pass an `accelerate.DataLoaderConfiguration` instead: 
dataloader_config = DataLoaderConfiguration(dispatch_batches=None, split_batches=False, even_batches=True, use_seedable_sampler=True)
  warnings.warn(


In [ ]:
gflan_test_results = trainer_gflan_test.evaluate()

In [ ]:
gflan_test_results

{'eval_loss': 0.46690666675567627,
 'eval_f1': 0.0,
 'eval_Accuracy': 0.04050322184719239,
 'eval_runtime': 25.231,
 'eval_samples_per_second': 129.167,
 'eval_steps_per_second': 16.171}

In [ ]:
wandb.log({"test_accuracy": gflan_test_results["eval_Accuracy"], "test_loss": gflan_test_results["eval_loss"], "test_f1": gflan_test_results["eval_f1"]})

In [ ]:
gflan_test_output = trainer_gflan_test.predict(tokenized_dataset_test_gflan)
gflan_test_output

PredictionOutput(predictions=(array([[ 0.40528938, -2.4906216 ,  1.1295809 , ..., -0.15976898,
        -3.2052555 , -4.980635  ],
       [ 0.6588483 , -2.818307  ,  0.8934274 , ..., -0.413132  ,
        -3.7578309 , -5.032128  ],
       [ 1.4906839 , -3.701061  ,  1.7147295 , ...,  1.0162092 ,
        -3.9697008 , -5.3457966 ],
       ...,
       [ 3.3714685 , -3.4026535 ,  2.8516953 , ..., -2.1773117 ,
        -3.7083304 , -4.293614  ],
       [-4.539358  , -2.1267896 , -4.3656974 , ..., -3.5838168 ,
        -2.9435031 , -1.9713787 ],
       [-3.0435128 , -1.1979388 , -3.2789989 , ..., -3.3666155 ,
        -2.3968062 , -2.7175508 ]], dtype=float32), array([[[-0.06766578,  0.1539964 , -0.10013093, ...,  0.02316707,
          0.12443022, -0.06255454],
        [-0.02824328,  0.12430257, -0.19660501, ...,  0.05585163,
          0.14471573, -0.18388589],
        [-0.06920994,  0.07203022, -0.11024038, ..., -0.01347893,
          0.03717359, -0.06698028],
        ...,
        [-0.08785165, 

In [ ]:
gflan_test_preds = gflan_test_output.predictions

In [ ]:
gflan_test_preds = torch.tensor(gflan_test_preds[0])

##Predictions to Probablities

In [ ]:
def coversion(predictions, threshold=0.5):
    sigmoid = torch.nn.Sigmoid()
    probs = sigmoid(torch.Tensor(predictions))
    results = np.zeros(probs.shape)
    results[np.where(probs >= threshold)] = 1
    return results

In [ ]:
gflan_test_output_final = coversion(gflan_test_preds)
(gflan_test_output_final)

array([[1., 0., 1., ..., 0., 0., 0.],
       [1., 0., 1., ..., 0., 0., 0.],
       [1., 0., 1., ..., 1., 0., 0.],
       ...,
       [1., 0., 1., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.]])

In [ ]:
df_test = pd.read_csv(datafolder/"emoticons_test.csv")

In [ ]:
df_test.columns

Index(['ID', 'Tweet', 'anger', 'anticipation', 'disgust', 'fear', 'joy',
       'love', 'optimism', 'pessimism', 'sadness', 'surprise', 'trust'],
      dtype='object')

In [ ]:
df_test[['anger', 'anticipation', 'disgust', 'fear', 'joy', 'love',
       'optimism', 'pessimism', 'sadness', 'surprise', 'trust']] = gflan_test_output_final

In [ ]:
df_test.drop(["Tweet"],axis =1, inplace = True)

In [ ]:
df_test.to_csv(basefolder/"Class_8/Home_Work_6/Submissions/gdlan-test-0.csv")

##Inference

In [ ]:
mapped_labels =[]
for prediction_row in gflan_test_output_final:
    mapped_row = [label for label, pred in zip(labels, prediction_row) if pred]
    mapped_labels.append(mapped_row)

In [ ]:
from pprint import pprint
pprint(mapped_labels[:10])

[['anger', 'disgust', 'fear'],
 ['anger', 'disgust'],
 ['anger', 'disgust', 'sadness'],
 [],
 ['fear', 'pessimism', 'sadness'],
 ['disgust'],
 ['joy'],
 ['joy', 'optimism'],
 ['joy', 'love', 'optimism'],
 ['fear', 'pessimism', 'sadness']]
